### Ensembles with pipeline and column transformer

In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [150]:
df=sns.load_dataset('tips')

In [151]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [152]:
# predict what's the time if its lunch or dinner>>time target variabel

df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [153]:
# EDA>> subjective
# encoding, missing valuse treatement , scalling>>>> automate this
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [154]:
# since time is nomianal variable, we will use label encoder
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [155]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [156]:
df.time.unique()  
# dinner >> 1
# lunch >> 0

array([0, 1])

In [157]:
df.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [158]:
X=df.drop('time', axis=1)
y=df['time']

In [159]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [160]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [161]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train ,y_test=train_test_split(X, y, test_size=0.20, random_state=1)

In [162]:
X_train

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
154,19.77,2.00,Male,No,Sun,4
167,31.71,4.50,Male,No,Sun,4
110,14.00,3.00,Male,No,Sat,2
225,16.27,2.50,Female,Yes,Fri,2
...,...,...,...,...,...,...
137,14.15,2.00,Female,No,Thur,2
72,26.86,3.14,Female,Yes,Sat,2
140,17.47,3.50,Female,No,Thur,2
235,10.07,1.25,Male,No,Sat,2


In [163]:
df.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [164]:
# handling the missing value
# data encoding
# feature scalling

# automat

from sklearn.impute import SimpleImputer   # for missing value
from sklearn.preprocessing import OneHotEncoder  # for encoding the data
from sklearn.preprocessing import StandardScaler   # for scalling

from sklearn.pipeline import Pipeline  # a sequence of data tranformer in order>> step1, step2, step3
from sklearn.compose import ColumnTransformer  # group all the pipline steps for each of the colum

In [165]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [166]:
cat_cols=['sex', 'smoker', 'day']
num_cols=['total_bill', 'tip', 'size']

In [167]:
# feature engineering automation using pipeline and columns transformer

num_pipeline=Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),
                             ('scaling', StandardScaler()) ])  # median beause outlier tretment nhi kiye h

cat_pipeline=Pipeline(steps=[('imputation', SimpleImputer(strategy='most_frequent')),
                              ('encoding', OneHotEncoder())]) 

In [168]:
preprocessor=ColumnTransformer([('num_pipeline', num_pipeline, num_cols), ('cat_pipeline', cat_pipeline, cat_cols)])

In [169]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [170]:
X_train

array([[-0.28611937, -1.47443803, -0.57766863, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.02695905, -0.71612531,  1.47042924, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.3716196 ,  1.19880579,  1.47042924, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.23206267,  0.43283335, -0.57766863, ...,  0.        ,
         0.        ,  1.        ],
       [-1.06543688, -1.29060464, -0.57766863, ...,  1.        ,
         0.        ,  0.        ],
       [-0.29287646,  0.1034652 ,  0.44638031, ...,  1.        ,
         0.        ,  0.        ]])

In [171]:
X_test

array([[-1.85376383, -1.48209775, -1.60171757,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.08453291,  0.04984713, -0.57766863,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.79501474,  0.36389583,  0.44638031,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.59356688, -0.33313909, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.18349826,  0.04984713, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-1.32783714, -1.14506988, -0.57766863,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.   

In [172]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

models={'support vector classifier': SVC(), 
        'DT classifier': DecisionTreeClassifier(),
        'Log Regression': LogisticRegression()}

In [173]:
len(models)

3

In [174]:
models.values()

dict_values([SVC(), DecisionTreeClassifier(), LogisticRegression()])

In [175]:
list(models.values())

[SVC(), DecisionTreeClassifier(), LogisticRegression()]

In [176]:
models.keys()

dict_keys(['support vector classifier', 'DT classifier', 'Log Regression'])

In [177]:
list(models.keys())

['support vector classifier', 'DT classifier', 'Log Regression']

In [178]:
from sklearn.metrics import accuracy_score
def model_train_eval(X_train,y_train, X_test, y_test, models): 
    evaluation={}
    for i in range(len(models)): 
        model=list(models.values())[i]
        model.fit(X_train, y_train)
        y_pred=model.predict(X_test)
        model_score=accuracy_score(y_test, y_pred)
        evaluation[list(models.keys())[i]]=model_score
    return evaluation

In [179]:
model_train_eval(X_train, y_train, X_test, y_test, models)

{'support vector classifier': 0.9183673469387755,
 'DT classifier': 0.8979591836734694,
 'Log Regression': 0.9183673469387755}

In [180]:
from sklearn.ensemble import RandomForestClassifier

In [181]:
rf=RandomForestClassifier()
rf

RandomForestClassifier()

In [182]:
X_train

array([[-0.28611937, -1.47443803, -0.57766863, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.02695905, -0.71612531,  1.47042924, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.3716196 ,  1.19880579,  1.47042924, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.23206267,  0.43283335, -0.57766863, ...,  0.        ,
         0.        ,  1.        ],
       [-1.06543688, -1.29060464, -0.57766863, ...,  1.        ,
         0.        ,  0.        ],
       [-0.29287646,  0.1034652 ,  0.44638031, ...,  1.        ,
         0.        ,  0.        ]])

In [183]:
X_test

array([[-1.85376383, -1.48209775, -1.60171757,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.08453291,  0.04984713, -0.57766863,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.79501474,  0.36389583,  0.44638031,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.59356688, -0.33313909, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.18349826,  0.04984713, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-1.32783714, -1.14506988, -0.57766863,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.   

In [184]:
from sklearn.model_selection import RandomizedSearchCV

params={'max_depth':[1,2,3,4,5,10,None],
        'n_estimators': [50,100,200,300],  # by default 100>> how many model(dt) you want to create
        'criterion': ['gini', 'entropy']
        }

In [185]:
params

{'max_depth': [1, 2, 3, 4, 5, 10, None],
 'n_estimators': [50, 100, 200, 300],
 'criterion': ['gini', 'entropy']}

In [186]:
clf=RandomizedSearchCV(rf, param_distributions=params, cv=5, verbose=3, scoring='accuracy')

In [187]:
clf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=2, n_estimators=200;, score=0.923 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=2, n_estimators=200;, score=0.974 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=2, n_estimators=200;, score=1.000 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=2, n_estimators=200;, score=1.000 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=2, n_estimators=200;, score=1.000 total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=1, n_estimators=50;, score=0.923 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=1, n_estimators=50;, score=0.974 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=1, n_estimators=50;, score=0.795 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=1, n_estimators=50;, score=0.949 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=1, n_estimators=50;, score=0.949 total time=  

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 2, 3, 4, 5, 10, None],
                                        'n_estimators': [50, 100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [188]:
clf.best_params_

{'n_estimators': 200, 'max_depth': 2, 'criterion': 'entropy'}

In [189]:
clf.best_score_

0.9794871794871796

In [190]:
y_pred=clf.predict(X_test)

In [191]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9183673469387755
              precision    recall  f1-score   support

           0       0.93      0.97      0.95        38
           1       0.89      0.73      0.80        11

    accuracy                           0.92        49
   macro avg       0.91      0.85      0.87        49
weighted avg       0.92      0.92      0.92        49



In [ ]:
# 0.9183673469387755 this accuracy is come form hyperparameter tuning and randomisec search cv

In [192]:
# we can also do this ways: 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

models={'support vector classifier': SVC(), 
        'DT classifier': DecisionTreeClassifier(),
        'Log Regression': LogisticRegression(),
        'Random Forest': RandomForestClassifier()}

In [193]:
from sklearn.metrics import accuracy_score
def model_train_eval(X_train,y_train, X_test, y_test, models): 
    evaluation={}
    for i in range(len(models)): 
        model=list(models.values())[i]
        model.fit(X_train, y_train)
        y_pred=model.predict(X_test)
        model_score=accuracy_score(y_test, y_pred)
        evaluation[list(models.keys())[i]]=model_score
    return evaluation

In [194]:
model_train_eval(X_train, y_train, X_test, y_test, models)

{'support vector classifier': 0.9183673469387755,
 'DT classifier': 0.9183673469387755,
 'Log Regression': 0.9183673469387755,
 'Random Forest': 0.8979591836734694}

In [195]:
# Random Forest': 0.897959183673469 this score is came from by default parameter

In [ ]:
# homework: use total_bill as target variable and make it a regresision problem and automat the fe, tarainign, tesing  and also use random forest regression with MLE , SVR models